In [12]:
from torch import nn
import torch

In [3]:
seq = nn.Sequential(
    nn.Conv2d(2, 2, kernel_size=(3, 3)),
    nn.Conv2d(2, 2, kernel_size=(3, 3)),
    nn.Conv2d(2, 2, kernel_size=(3, 3)),
    nn.Conv2d(2, 2, kernel_size=(3, 3)),
    nn.Conv2d(2, 2, kernel_size=(3, 3))
)

In [4]:
len(seq)

5

In [ ]:
seq.load_state_dict

In [8]:
cs = [3, 13, 15, 12]
layers = [nn.Conv2d(cs[i], cs[i+1], kernel_size=(3, 3))
          for i in range(len(cs) - 1)]
nn.Sequential(*layers)
    

Sequential(
  (0): Conv2d(3, 13, kernel_size=(3, 3), stride=(1, 1))
  (1): Conv2d(13, 15, kernel_size=(3, 3), stride=(1, 1))
  (2): Conv2d(15, 12, kernel_size=(3, 3), stride=(1, 1))
)

In [43]:
from torch import nn


class LogisticRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_layer = nn.Linear(3 * 32 * 32, 10)

    def forward(self, x):
        x = self.hidden_layer(x.view(-1, 3 * 32 * 32))
        return x


class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_layers = nn.Sequential(
            nn.Linear(3 * 32 * 32, 128),
            nn.Sigmoid(),
            nn.Linear(128, 128),
            nn.Sigmoid(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.hidden_layers(x.view(-1, 3 * 32 * 32))
        return x


class ConvNet1(nn.Module):
    def __init__(self,
                 n_classes=10,
                 channel_sequence=[32, 64, 64, 64],
                 fc_type='linear'):
        super().__init__()

        cs = [3] + channel_sequence
        layers = [self._block(cs[i], cs[i+1]) for i in range(len(channel_sequence))]
        self.convs = nn.Sequential(*layers)

        self.last_conv_res = 32 // 2**len(self.convs)
        assert self.last_conv_res != 0

        self.pre_fc_size = channel_sequence[-1] * self.last_conv_res**2

        if fc_type == 'linear':
            self.fc = nn.Linear(self.pre_fc_size, n_classes)
        elif fc_type == 'nonlinear':
            self.fc = nn.Sequential(
                nn.Linear(self.pre_fc_size, self.pre_fc_size),
                nn.ReLU(inplace=True),
                nn.Linear(self.pre_fc_size, n_classes)
            )
        elif fc_type == 'nonlinear_dropout':
            self.fc = nn.Sequential(
                nn.Linear(self.pre_fc_size, self.pre_fc_size),
                nn.Dropout(p=0.5),
                nn.ReLU(inplace=True),
                nn.Linear(self.pre_fc_size, n_classes)
            )
        else:
            raise Exception("No fc_type: " + fc_type)

    def _block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
        )

    def forward(self, x):
        x = self.convs(x)
        x = self.fc(x.view(-1, self.pre_fc_size))
        return x

# here we want to only use convolutional models, so we won'y use linear regression on MLP

MODELS = {
    'convnet1': ConvNet1
}


In [17]:
z = ConvNet1()

In [18]:
z

ConvNet1(
  (convs): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (3): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (fc): Linear(in_features=256, out_features=10, bias=True)
)

In [19]:
z(torch.rand(12, 3, 32, 32))

tensor([[-5.1719e-02,  6.1140e-02, -3.9694e-02,  3.2859e-03,  4.7765e-02,
         -3.7687e-02, -1.8440e-02,  2.2842e-02,  2.2674e-02,  6.3018e-02],
        [-5.3767e-02,  6.0321e-02, -3.5300e-02,  5.3731e-04,  4.8598e-02,
         -3.4665e-02, -1.1033e-02,  1.8304e-02,  2.2516e-02,  6.4102e-02],
        [-4.9139e-02,  5.6575e-02, -3.9400e-02, -1.0377e-03,  4.5235e-02,
         -3.5975e-02, -1.5631e-02,  2.0218e-02,  2.5695e-02,  6.5166e-02],
        [-5.1037e-02,  5.5594e-02, -3.7847e-02,  1.3498e-03,  5.0618e-02,
         -3.3166e-02, -1.7107e-02,  2.0209e-02,  2.4526e-02,  6.1187e-02],
        [-4.7295e-02,  5.8555e-02, -3.6046e-02,  1.2554e-03,  4.7957e-02,
         -3.8134e-02, -1.7631e-02,  2.4532e-02,  2.2807e-02,  6.3291e-02],
        [-5.1131e-02,  5.5734e-02, -3.9104e-02,  1.5010e-03,  4.8751e-02,
         -3.8461e-02, -1.5809e-02,  1.8470e-02,  2.5052e-02,  6.8164e-02],
        [-5.0941e-02,  5.6823e-02, -3.9106e-02, -8.4635e-05,  4.6898e-02,
         -3.3268e-02, -1.2134e-0

In [15]:
z.last_conv_res

8

In [20]:
import torch
from torch.utils import data
from torchvision import datasets, transforms


def get_image_datasets(data_dir):
    # transformations to be applied to all images
    transform = transforms.Compose([
        transforms.ToTensor(),                                   # transform to tensor of values in range [0., 1.]
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # download CIFAR10 train and validation datasets
    image_datasets = {
        'train':
        datasets.CIFAR10(root=data_dir, train=True, download=True, transform=transform),
        'validation':
        datasets.CIFAR10(root=data_dir, train=False, download=True, transform=transform)
    }
    return image_datasets

In [21]:
get_image_datasets("./cifar_pytorch")

Files already downloaded and verified


{'train': Dataset CIFAR10
     Number of datapoints: 50000
     Split: train
     Root Location: ./cifar_pytorch
     Transforms (if any): Compose(
                              ToTensor()
                              Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                          )
     Target Transforms (if any): None, 'validation': Dataset CIFAR10
     Number of datapoints: 10000
     Split: test
     Root Location: ./cifar_pytorch
     Transforms (if any): Compose(
                              ToTensor()
                              Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                          )
     Target Transforms (if any): None}

In [22]:
z

ConvNet1(
  (convs): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (3): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (fc): Linear(in_features=256, out_features=10, bias=True)
)

In [24]:
z.convs[:2]

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
)

In [45]:
class ModelLeftPart(nn.Module):
    def __init__(self, original_model, cut_at):
        super().__init__()
        self.convs = original_model.convs[:cut_at]

    def forward(self, x):
        x = self.convs(x)
        return x

class ModelRightPart(nn.Module):
    def __init__(self, original_model, cut_at):
        super().__init__()
        self.convs = original_model.convs[cut_at:]
        self.fc = original_model.fc

    def forward(self, x):
        x = self.convs(x)
        x = self.fc(x.view(x.size(0), -1))
        return x

In [34]:
model1 = ConvNet1()

In [35]:
model1

ConvNet1(
  (convs): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (3): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (fc): Linear(in_features=256, out_features=10, bias=True)
)

In [41]:
ModelLeftPart(model1, 1)

ModelLeftPart(
  (convs): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
  )
)

In [44]:
model1

ConvNet1(
  (convs): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (3): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (fc): Linear(in_features=256, out_features=10, bias=True)
)

In [46]:
ModelRightPart(model1, 1)

ModelRightPart(
  (convs): Sequential(
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (3): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (fc): Linear(in_features=256, out_features=10, bias=True)
)

In [49]:
random_input = torch.randn(1, 3, 32, 32)
model1(random_input)

tensor([[-0.0515,  0.1779, -0.1048,  0.0732, -0.0543,  0.0521, -0.1128,  0.0309,
         -0.0094, -0.1094]], grad_fn=<AddmmBackward>)

In [51]:
m1l = ModelLeftPart(model1, 1)
m1r = ModelRightPart(model1, 1)

m1r(m1l(random_input))

tensor([[-0.0515,  0.1779, -0.1048,  0.0732, -0.0543,  0.0521, -0.1128,  0.0309,
         -0.0094, -0.1094]], grad_fn=<AddmmBackward>)

In [52]:
model1.convs.do[0].in_channels

AttributeError: 'Sequential' object has no attribute 'in_channels'

In [55]:
list(model1.convs.modules())[0]

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (3): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
)

In [59]:
model1.convs[0][0].in_channels

3

In [61]:
model1.convs.__class__

torch.nn.modules.container.Sequential